In [69]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import pandas as pd
import konlpy

data = pd.read_csv("sample_text.csv", header=None, names=['text'])
data.head()

,text
0,인공지능 또는 AI는 인간의 학습능력 추론능력 지각능력 그외에 인공적으로 구현한 컴...
1,하나의 인프라 기술이기도 하다
2,인간을 포함한 동물이 갖고 있는 지능 즉 natural intelligence와는 ...
3,지능을 갖고 있는 기능을 갖춘 컴퓨터 시스템이며 인간의 지능을 기계 등에 인공적으로...
4,일반적으로 범용 컴퓨터에 적용한다고 가정한다


In [70]:
data['text'] = data['text'].str.replace(",", "").replace(".", "")
data['morphs'] = data['text'].map(lambda x: tokenizer.morphs(x))
data

,text,morphs
0,인공지능 또는 AI는 인간의 학습능력 추론능력 지각능력 그외에 인공적으로 구현한 컴...,"[인공지능, 또는, AI, 는, 인간, 의, 학습, 능력, 추론, 능력, 지각, 능..."
1,하나의 인프라 기술이기도 하다,"[하나, 의, 인프라, 기술, 이, 기, 도, 하, 다]"
2,인간을 포함한 동물이 갖고 있는 지능 즉 natural intelligence와는 ...,"[인간, 을, 포함, 한, 동물, 이, 갖, 고, 있, 는, 지능, 즉, natur..."
3,지능을 갖고 있는 기능을 갖춘 컴퓨터 시스템이며 인간의 지능을 기계 등에 인공적으로...,"[지능, 을, 갖, 고, 있, 는, 기능, 을, 갖춘, , 컴퓨터, 시스템, 이,..."
4,일반적으로 범용 컴퓨터에 적용한다고 가정한다,"[일반, 적, 으로, 범용, 컴퓨터, 에, 적용, 한다고, 가정, 한다]"
5,이 용어는 또한 그와 같은 지능을 만들 수 있는 방법론이나 실현 가능성 등을 연구하...,"[이, 용어, 는, 또한, 그, 와, 같, 은, 지능, 을, 만들, 수, 있, 는,..."


In [71]:
words = set()
data['morphs'].map(lambda x: words.update(set(x)))

word_to_index = {w: i + 4 for i, w in enumerate(words)}
word_to_index["PAD"] = 0
word_to_index["UNK"] = 1
word_to_index["SOS"] = 2
word_to_index["EOS"] = 3

index_to_word = {}
for key, value in word_to_index.items():
    index_to_word[value] = key

In [72]:
chars = set([w_i for w in word_set for w_i in w])
chars = sorted(list(chars))

char_to_index = {c: i + 2 for i, c in enumerate(chars)}
char_to_index["PAD"] = 0
char_to_index["UNK"] = 1

index_to_char = {}
for key, value in char_to_index.items():
    index_to_char[value] = key

In [132]:
max_len_char = 10
max_len_word = 50

def integer_coding(sentences, max_len_char, max_len_word):
    word_data = []
    
    for i, word in enumerate(sentences):
        word_data.append(word_to_index[word])
        char_indice = [[char_to_index[char] for char in word]]
        char_indice = pad_sequences(char_indice, maxlen=max_len_char, padding='post', value=0)

        for chars_of_token in char_indice:
            if len(chars_of_token) > max_len_char:
                continue
        
        if i == 0:
            char_data = char_indice
        else:
            char_data = np.vstack([char_data, char_indice])
    
    pad_char_data = max_len_word - char_data.shape[0]
    
    char_data = np.pad(char_data, ((0, pad_char_data), (0, 0)), 'constant', constant_values=0)
    word_data = pad_sequences([word_data], maxlen=max_len_word, padding='post', value=0)
    
    return word_data, char_data

In [133]:
data.loc[0, "morphs"]

['인공지능',
 '또는',
 'AI',
 '는',
 '인간',
 '의',
 '학습',
 '능력',
 '추론',
 '능력',
 '지각',
 '능력',
 '그',
 '외',
 '에',
 '인공',
 '적',
 '으로',
 '구현',
 '한',
 '컴퓨터',
 '프로그램',
 '또는',
 '이',
 '를',
 '포함',
 '한',
 '컴퓨터',
 '시스템',
 '이',
 '다']

In [134]:
X_word_data, X_char_data = integer_coding(data.loc[0, "morphs"], max_len_char, max_len_word)
X_char_data

array([[73, 24, 79, 32,  0,  0,  0,  0,  0,  0],
       [38, 31,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 2,  3,  0,  0,  0,  0,  0,  0,  0,  0],
       [31,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [73, 17,  0,  0,  0,  0,  0,  0,  0,  0],
       [71,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [90, 58,  0,  0,  0,  0,  0,  0,  0,  0],
       [32, 41,  0,  0,  0,  0,  0,  0,  0,  0],
       [80, 43,  0,  0,  0,  0,  0,  0,  0,  0],
       [32, 41,  0,  0,  0,  0,  0,  0,  0,  0],
       [79, 16,  0,  0,  0,  0,  0,  0,  0,  0],
       [32, 41,  0,  0,  0,  0,  0,  0,  0,  0],
       [27,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [66,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [63,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [73, 24,  0,  0,  0,  0,  0,  0,  0,  0],
       [76,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [68, 42,  0,  0,  0,  0,  0,  0,  0,  0],
       [26, 93,  0,  0,  0,  0,  0,  0,  0,  0],
       [91,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [83, 87, 84, 

In [135]:
X_word_data

array([[19, 43, 57, 27, 69,  7,  5, 51, 17, 51, 34, 51, 67, 59, 39, 40,
        33, 75, 55, 52,  9, 45, 43, 10, 46, 16, 52,  9, 74, 10, 70,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0]], dtype=int32)